In [83]:
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using Plots
using LinearAlgebra
using CSV
using StatsBase
using GLM
using Distributions
include("../utils.jl")


print_percentiles (generic function with 3 methods)

In [3]:
columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net, :ACWI_Net_PLN, :MwigTR, :WIG20TR, :B10YPL]
data_df = DataFrame(XLSX.readtable("../data/assets_data_03.24.xlsx", "Indexes_PLN"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
returns_pln = data_df |> 
            x -> TimeArray(x; timestamp = :Date) |> 
            x -> from(x, Date(2002,12,01)) |>
            x -> percentchange(x, :log)

255×7 TimeArray{Float64, 2, Date, Matrix{Float64}} 2003-01-31 to 2024-03-31
┌────────────┬────────────┬──────────────┬─────────────┬──────────────┬─────────
│            │ USA_Net    │ World_ex_USA │ EM_Net      │ ACWI_Net_PLN │ MwigTR ⋯
├────────────┼────────────┼──────────────┼─────────────┼──────────────┼─────────
│ 2003-01-31 │ -0.0248446 │   -0.0388471 │ -0.00442015 │   -0.0317079 │  -0.04 ⋯
│ 2003-02-28 │ -0.0159343 │   -0.0199094 │  -0.0284988 │   0.00510351 │ -0.001 ⋯
│ 2003-03-31 │ 0.00886423 │   -0.0194601 │  -0.0289739 │    0.0430687 │  -0.00 ⋯
│ 2003-04-30 │  0.0798764 │    0.0916379 │   0.0851925 │    0.0118412 │   0.03 ⋯
│ 2003-05-31 │  0.0517369 │    0.0601744 │   0.0690563 │    0.0300993 │   0.05 ⋯
│ 2003-06-30 │   0.011892 │    0.0240024 │   0.0551391 │    0.0670022 │   0.01 ⋯
│ 2003-07-31 │  0.0179915 │    0.0226828 │   0.0603263 │    0.0132941 │    0.1 ⋯
│ 2003-08-31 │  0.0181089 │    0.0254588 │   0.0647735 │    0.0495565 │    0.1 ⋯
│     ⋮      │     ⋮      │      

In [4]:
data_df = DataFrame(XLSX.readtable("../data/assets_data_03.24.xlsx", "Macro_month"; infer_eltypes=true))[:,[:Date, :WIBOR3m, :CPIPLM, :Y10PL, :MWig40_dy]]
dropmissing!(data_df, disallowmissing=true)
macro_pl = data_df |> x -> TimeArray(x; timestamp = :Date)  |> x -> from(x,Date(2003,01,01)) 

204×4 TimeArray{Float64, 2, Date, Matrix{Float64}} 2007-08-31 to 2024-07-31
┌────────────┬─────────┬────────┬─────────┬───────────┐
│            │ WIBOR3m │ CPIPLM │ Y10PL   │ MWig40_dy │
├────────────┼─────────┼────────┼─────────┼───────────┤
│ 2007-08-31 │  0.0503 │ -0.004 │ 0.05789 │   0.00668 │
│ 2007-09-30 │   0.051 │  0.008 │   0.057 │   0.00778 │
│ 2007-10-31 │  0.0521 │  0.006 │ 0.05514 │   0.00775 │
│ 2007-11-30 │  0.0556 │  0.007 │ 0.05777 │    0.0093 │
│ 2007-12-31 │   0.057 │  0.003 │ 0.05916 │    0.0095 │
│ 2008-01-31 │  0.0565 │  0.007 │ 0.05688 │     0.011 │
│ 2008-02-29 │  0.0594 │  0.004 │ 0.05973 │    0.0118 │
│ 2008-03-31 │  0.0615 │  0.004 │ 0.05971 │    0.0188 │
│     ⋮      │    ⋮    │   ⋮    │    ⋮    │     ⋮     │
│ 2024-01-31 │  0.0586 │  0.004 │ 0.05226 │   0.02643 │
│ 2024-02-29 │  0.0586 │  0.003 │ 0.05312 │   0.03451 │
│ 2024-03-31 │  0.0588 │  0.002 │ 0.05432 │   0.03081 │
│ 2024-04-30 │  0.0586 │  0.011 │ 0.05712 │   0.03272 │
│ 2024-05-31 │  0.0585 │  0.

In [89]:
columns = [:Date, :ACWI_Net, :USA_Net, :World_ex_USA, :EM_Net]
data_df = DataFrame(XLSX.readtable("../data/assets_data_03.24.xlsx", "Indexes"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
returns_usd = data_df |> 
            x -> TimeArray(x; timestamp = :Date) |> 
            x -> percentchange(x, :log) |>
            x -> from(x, Date(2003,01,01))

255×4 TimeArray{Float64, 2, Date, Matrix{Float64}} 2003-01-31 to 2024-03-31
┌────────────┬─────────────┬────────────┬──────────────┬─────────────┐
│            │ ACWI_Net    │ USA_Net    │ World_ex_USA │ EM_Net      │
├────────────┼─────────────┼────────────┼──────────────┼─────────────┤
│ 2003-01-31 │  -0.0298776 │ -0.0248446 │   -0.0388471 │ -0.00442015 │
│ 2003-02-28 │  -0.0181007 │ -0.0159343 │   -0.0199094 │  -0.0284988 │
│ 2003-03-31 │ -0.00432387 │ 0.00886423 │   -0.0194601 │  -0.0289739 │
│ 2003-04-30 │   0.0849273 │  0.0798764 │    0.0916379 │   0.0851925 │
│ 2003-05-31 │   0.0559049 │  0.0517369 │    0.0601744 │   0.0690563 │
│ 2003-06-30 │   0.0185506 │   0.011892 │    0.0240024 │   0.0551391 │
│ 2003-07-31 │   0.0216391 │  0.0179915 │    0.0226828 │   0.0603263 │
│ 2003-08-31 │   0.0231143 │  0.0181089 │    0.0254588 │   0.0647735 │
│     ⋮      │      ⋮      │     ⋮      │      ⋮       │      ⋮      │
│ 2023-09-30 │   -0.042229 │ -0.0483353 │   -0.0342956 │  -0.0264995 │
│

In [90]:
returns_summarystats(returns_pln, 12)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
USA_Net,0.1494,1.4788,0.0985,0.4065,-2.257,-0.7909,-0.1905,0.0391,1.8967,0.6593,0.1537
World_ex_USA,0.168,1.7129,0.0702,0.3971,-2.7986,-0.8513,-0.2294,0.1307,2.6166,0.4181,0.1466
EM_Net,0.2087,1.893,0.0842,0.553,-3.8371,-0.8236,-0.3169,0.1393,3.2059,0.4033,0.0995
ACWI_Net_PLN,0.121,1.2714,0.0879,0.3596,-1.2476,-0.3241,-0.1698,0.0111,0.6767,0.7263,0.0957
MwigTR,0.2181,2.403,0.1155,0.4835,-3.8898,-0.6628,-0.2993,0.1952,3.7443,0.5296,0.1475
WIG20TR,0.2199,2.2594,0.0705,0.594,-3.2019,-0.3645,-0.4047,0.0087,1.0945,0.3205,0.1144
B10YPL,0.0895,1.5024,0.0444,0.206,-1.0138,0.0388,-0.1158,0.0204,3.1919,0.4963,0.0604


In [91]:
returns_summarystats(returns_usd,12)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
ACWI_Net,0.1566,1.3949,0.0859,0.4036,-2.65,-0.9343,-0.1938,0.0946,2.7364,0.5487,0.1571
USA_Net,0.1494,1.4788,0.0985,0.4065,-2.257,-0.7909,-0.1905,0.0391,1.8967,0.6593,0.1537
World_ex_USA,0.168,1.7129,0.0702,0.3971,-2.7986,-0.8513,-0.2294,0.1307,2.6166,0.4181,0.1466
EM_Net,0.2087,1.893,0.0842,0.553,-3.8371,-0.8236,-0.3169,0.1393,3.2059,0.4033,0.0995


In [92]:
ols = lm(@formula(USA_Net ~ MwigTR), returns_pln)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

USA_Net ~ 1 + MwigTR

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)  0.00367412  0.00198719   1.85    0.0656  -0.00023942  0.00758766
MwigTR       0.470963    0.0312627   15.06    <1e-36   0.409395    0.532531
─────────────────────────────────────────────────────────────────────────────
0.47285673857874355


In [93]:
ols = lm(@formula(MwigTR ~ USA_Net), returns_pln)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

MwigTR ~ 1 + USA_Net

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)  0.00138433   0.0029197   0.47    0.6358  -0.00436569  0.00713434
USA_Net      1.00402      0.0666472  15.06    <1e-36   0.872767    1.13528
─────────────────────────────────────────────────────────────────────────────
0.4728567385787438


In [94]:
returns_lag = merge(returns_pln, TimeSeries.lead(returns_pln,1))
ols = lm(@formula(MwigTR ~  USA_Net_1 + USA_Net), returns_lag)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

MwigTR ~ 1 + USA_Net_1 + USA_Net

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)  0.000574091  0.00296783   0.19    0.8468  -0.00527092  0.00641911
USA_Net_1    0.10924      0.0666639    1.64    0.1025  -0.0220515   0.240532
USA_Net      1.00065      0.0666232   15.02    <1e-36   0.869436    1.13186
──────────────────────────────────────────────────────────────────────────────
0.47876745733429493


In [95]:
ols = lm(@formula(World_ex_USA ~ MwigTR), returns_pln)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

World_ex_USA ~ 1 + MwigTR

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)  0.000606901  0.00217528   0.28    0.7805  -0.00367706  0.00489086
MwigTR       0.54516      0.0342217   15.93    <1e-39   0.477764    0.612555
──────────────────────────────────────────────────────────────────────────────
0.5007618342522057


In [96]:
ols = lm(@formula(EM_Net ~ MwigTR), returns_pln)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

EM_Net ~ 1 + MwigTR

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)  0.000975548  0.00288701   0.34    0.7357  -0.00471008  0.00666118
MwigTR       0.627412     0.0454187   13.81    <1e-31   0.537966    0.716859
──────────────────────────────────────────────────────────────────────────────
0.42995612230004543


In [97]:
columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net]
data_df = DataFrame(XLSX.readtable("../data/assets_data_03.24.xlsx", "Indexes"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
returns_usd = data_df |> 
            x -> TimeArray(x; timestamp = :Date) |> 
            x -> percentchange(x, :log)

435×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-01-31 to 2024-03-31
┌────────────┬────────────┬──────────────┬─────────────┐
│            │ USA_Net    │ World_ex_USA │ EM_Net      │
├────────────┼────────────┼──────────────┼─────────────┤
│ 1988-01-31 │  0.0410122 │    0.0153553 │   0.0935834 │
│ 1988-02-29 │  0.0401109 │    0.0640862 │   0.0031132 │
│ 1988-03-31 │ -0.0349774 │    0.0592025 │     0.10175 │
│ 1988-04-30 │ 0.00818061 │    0.0140195 │   0.0517417 │
│ 1988-05-31 │ 0.00760205 │   -0.0326806 │   0.0292525 │
│ 1988-06-30 │  0.0445733 │   -0.0229028 │   0.0331365 │
│ 1988-07-31 │ -0.0040323 │    0.0288082 │  -0.0260317 │
│ 1988-08-31 │  -0.035096 │   -0.0667177 │  -0.0274362 │
│     ⋮      │     ⋮      │      ⋮       │      ⋮      │
│ 2023-09-30 │ -0.0483353 │   -0.0342956 │  -0.0264995 │
│ 2023-10-31 │ -0.0235616 │   -0.0431263 │   -0.039629 │
│ 2023-11-30 │  0.0895259 │    0.0898198 │   0.0769883 │
│ 2023-12-31 │  0.0456572 │    0.0532219 │   0.0383487 │
│ 2024-01-31

In [98]:
data = merge(returns_pln[:MwigTR], returns_usd[:USA_Net])
ols = lm(@formula(USA_Net ~ MwigTR), data)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

USA_Net ~ 1 + MwigTR

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)  0.00367412  0.00198719   1.85    0.0656  -0.00023942  0.00758766
MwigTR       0.470963    0.0312627   15.06    <1e-36   0.409395    0.532531
─────────────────────────────────────────────────────────────────────────────
0.47285673857874166


In [99]:
# number of month wig20 > mwig40

n_wig20Gr = values(returns_pln[:WIG20TR] .> returns_pln[:MwigTR])  |> count
n_wig20Gr / length(returns_pln)


0.47058823529411764

In [100]:
plot(rf)

UndefVarError: UndefVarError: `rf` not defined

In [101]:
bond_excess = returns_pln[:B10YPL] .- rf ./ 23 |>
    x -> from(x, Date(2015,01,01)) |>
    x -> to(x,  Date(2019,12,31)) 

returns_summarystats(bond_excess,12)

UndefVarError: UndefVarError: `rf` not defined

In [102]:
variables= merge(returns_pln[:USA_Net, :MwigTR], TimeSeries.lead(returns_pln[:USA_Net, :MwigTR]), macro_pl[:WIBOR3m, :CPIPLM, :MWig40_dy], TimeSeries.lead(macro_pl[:WIBOR3m, :CPIPLM, :MWig40_dy], 1), log.(TimeSeries.lead(macro_pl[:MWig40_dy],1)))
colnames(variables)

11-element Vector{Symbol}:
 :USA_Net
 :MwigTR
 :USA_Net_1
 :MwigTR_1
 :WIBOR3m
 :CPIPLM
 :MWig40_dy
 :WIBOR3m_1
 :CPIPLM_1
 :MWig40_dy_1
 :MWig40_dy_2

In [103]:
ols = lm(@formula(MwigTR ~ MWig40_dy_1), variables)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

MwigTR ~ 1 + MWig40_dy_1

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)   0.0156113  0.00978799   1.59    0.1123  -0.00369137   0.034914
MWig40_dy_1  -0.543517   0.385743    -1.41    0.1604  -1.30423      0.217199
────────────────────────────────────────────────────────────────────────────
0.009977192877162144


In [104]:
ols = lm(@formula(MwigTR ~ MWig40_dy_2), variables)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

MwigTR ~ 1 + MWig40_dy_2

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)   Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)  -0.0517075  0.0299636   -1.73    0.0860  -0.110798   0.00738299
MWig40_dy_2  -0.0139799  0.00754005  -1.85    0.0652  -0.0288495  0.000889673
─────────────────────────────────────────────────────────────────────────────
0.0171506453904694


In [105]:
ols = lm(@formula(MwigTR ~ MWig40_dy_1 + WIBOR3m_1 + CPIPLM_1), variables)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

MwigTR ~ 1 + MWig40_dy_1 + WIBOR3m_1 + CPIPLM_1

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)   0.0327325    0.012211   2.68    0.0080   0.00864989   0.0568152
MWig40_dy_1  -0.571674     0.38275   -1.49    0.1369  -1.32654      0.183187
WIBOR3m_1    -0.466034     0.221212  -2.11    0.0364  -0.90231     -0.029758
CPIPLM_1     -0.314899     0.86867   -0.36    0.7174  -2.02809      1.3983
─────────────────────────────────────────────────────────────────────────────
0.03658709151918704


In [106]:
ols = lm(@formula(MwigTR ~ MWig40_dy + WIBOR3m + CPIPLM), variables)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

MwigTR ~ 1 + MWig40_dy + WIBOR3m + CPIPLM

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)   0.0256318   0.0122461   2.09    0.0376   0.00147998   0.0497837
MWig40_dy    -0.270922    0.382717   -0.71    0.4799  -1.02572      0.483875
WIBOR3m      -0.539226    0.222151   -2.43    0.0161  -0.977353    -0.101098
CPIPLM        0.574589    0.866285    0.66    0.5079  -1.1339       2.28308
─────────────────────────────────────────────────────────────────────────────
0.03097387311577282


In [107]:
ols = lm(@formula(MwigTR ~ USA_Net_1 + MwigTR_1 + MWig40_dy_2 + WIBOR3m_1 + CPIPLM_1), variables)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

MwigTR ~ 1 + USA_Net_1 + MwigTR_1 + MWig40_dy_2 + WIBOR3m_1 + CPIPLM_1

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)   Lower 95%   Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)  -0.0346504  0.0301111   -1.15    0.2513  -0.0940394  0.0247386
USA_Net_1    -0.0304893  0.140162    -0.22    0.8280  -0.306936   0.245958
MwigTR_1      0.174875   0.107637     1.62    0.1059  -0.0374199  0.387171
MWig40_dy_2  -0.0131805  0.00746876  -1.76    0.0792  -0.0279114  0.00155035
WIBOR3m_1    -0.399705   0.222369    -1.80    0.0738  -0.838291   0.0388811
CPIPLM_1     -0.330082   0.860421    -0.38    0.7017  -2.02712    1.36695
───────────────────────────────────────────────────

In [108]:
ols = lm(@formula(MwigTR ~ USA_Net + MwigTR_1 + MWig40_dy_2 + WIBOR3m_1 + CPIPLM_1), variables)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

MwigTR ~ 1 + USA_Net + MwigTR_1 + MWig40_dy_2 + WIBOR3m_1 + CPIPLM_1

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)   Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)  -0.0140989   0.0201025   -0.70    0.4839  -0.0537476  0.0255499
USA_Net       0.965148    0.0620622   15.55    <1e-35   0.842741   1.08756
MwigTR_1      0.0923321   0.0474144    1.95    0.0529  -0.0011849  0.185849
MWig40_dy_2  -0.00337822  0.00501511  -0.67    0.5014  -0.0132697  0.00651324
WIBOR3m_1    -0.0927857   0.14922     -0.62    0.5348  -0.387097   0.201526
CPIPLM_1     -0.134408    0.573408    -0.23    0.8149  -1.26536    0.996543
─────────────────────────────────────────────

In [109]:
ols = lm(@formula(MwigTR ~ USA_Net + MwigTR_1 + MWig40_dy_2 + WIBOR3m + CPIPLM), variables)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

MwigTR ~ 1 + USA_Net + MwigTR_1 + MWig40_dy_2 + WIBOR3m + CPIPLM

Coefficients:
───────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)     Lower 95%   Upper 95%
───────────────────────────────────────────────────────────────────────────────
(Intercept)  -0.0149977   0.0201186   -0.75    0.4569  -0.0546783    0.0246828
USA_Net       0.96351     0.0621295   15.51    <1e-35   0.84097      1.08605
MwigTR_1      0.0934285   0.0472891    1.98    0.0496   0.000158668  0.186698
MWig40_dy_2  -0.00354078  0.00500684  -0.71    0.4803  -0.0134159    0.00633437
WIBOR3m      -0.125906    0.1489      -0.85    0.3988  -0.419587     0.167775
CPIPLM        0.324445    0.569833     0.57    0.5698  -0.799454     1.44834
────────────────────────────────

In [110]:
ols = lm(@formula(MwigTR ~ MwigTR_1), variables)
println(ols)
println(r2(ols))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

MwigTR ~ 1 + MwigTR_1

Coefficients:
───────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error     t  Pr(>|t|)    Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)  0.00257948  0.00430802  0.60    0.5500  -0.00591626  0.0110752
MwigTR_1     0.188414    0.0701444   2.69    0.0078   0.0500837   0.326744
───────────────────────────────────────────────────────────────────────────
0.0353307523248787


In [111]:
round.(cor(values(variables[:MwigTR, :USA_Net, :WIBOR3m, :CPIPLM, :MWig40_dy])), digits=2)

5×5 Matrix{Float64}:
  1.0    0.76  -0.16   0.0   -0.04
  0.76   1.0   -0.16  -0.01  -0.07
 -0.16  -0.16   1.0    0.27  -0.05
  0.0   -0.01   0.27   1.0    0.02
 -0.04  -0.07  -0.05   0.02   1.0

#### mwig dy eda

In [112]:
CSV.File("../data/mwig40price_dy.csv", delim=';')

70-element CSV.File:
 CSV.Row: (Date = String15("30.09.2007"), Mwig = String7("4568,14"), mWig_dy = String7("0,78%"), d = String15("35,5401292"), Column5 = String15("#ARG!"))
 CSV.Row: (Date = String15("31.12.2007"), Mwig = String7("4028,37"), mWig_dy = String7("0,95%"), d = String15("38,269515"), Column5 = String15("7,40%"))
 CSV.Row: (Date = String15("31.03.2008"), Mwig = String7("3363,11"), mWig_dy = String7("1,88%"), d = String15("63,226468"), Column5 = String15("50,21%"))
 CSV.Row: (Date = String15("30.06.2008"), Mwig = String7("2649,88"), mWig_dy = String7("0,97%"), d = String15("25,703836"), Column5 = String15("-90,01%"))
 CSV.Row: (Date = String15("30.09.2008"), Mwig = String7("2252,23"), mWig_dy = String7("1,61%"), d = String15("36,260903"), Column5 = String15("34,41%"))
 CSV.Row: (Date = String15("31.12.2008"), Mwig = String7("1511,27"), mWig_dy = String7("2,53%"), d = String15("38,235131"), Column5 = String15("5,30%"))
 CSV.Row: (Date = String15("31.03.2009"), Mwig = String7

In [113]:
mwig = TimeArray(CSV.File("../data/mwig40price_dy.csv", delim=';'), timestamp = :Date)

MethodError: MethodError: no method matching TimeArray(::Vector{Union{Missing, String15}}, ::Matrix{Union{Missing, String15}}, ::Vector{Symbol}, ::CSV.File; unchecked::Bool)

Closest candidates are:
  TimeArray(!Matched::AbstractVector{D}, ::AbstractArray{T, N}, ::Vector{Symbol}, ::Any; args...) where {T, N, D<:TimeType}
   @ TimeSeries C:\Users\matsz\.julia\packages\TimeSeries\zx7sJ\src\timearray.jl:89
  TimeArray(!Matched::D, ::AbstractArray{T, N}, ::Vector{Symbol}, ::Any; args...) where {T, N, D<:TimeType}
   @ TimeSeries C:\Users\matsz\.julia\packages\TimeSeries\zx7sJ\src\timearray.jl:94
  TimeArray(!Matched::AbstractVector{D}, ::AbstractArray{T, N}, !Matched::Vector{S}, ::Any; args...) where {T, N, D<:TimeType, S<:AbstractString}
   @ TimeSeries deprecated.jl:103
  ...


In [114]:
readtimearray("../data/mwig40price_dy.csv"; delim=';', format="dd/mm/yyyy")

ArgumentError: ArgumentError: Unable to parse date time. Expected directive Delim(/) at char 3

In [126]:
columns = [:Date, :USDPLN]
data_df = DataFrame(XLSX.readtable("../data/assets_data_03.24.xlsx", "Macro_month"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
usdpln = data_df |> 
            x -> TimeArray(x; timestamp = :Date) |>             
            x -> percentchange(x, :log)

returns_summarystats(usdpln, 12)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
USDPLN,0.1288,1.998,0.0045,0.2813,-1.1283,0.6829,-0.2977,0.061,1.7837,0.0351,-0.0231


In [124]:
returns_pln[:USA_Net]


255×1 TimeArray{Float64, 1, Date, Vector{Float64}} 2003-01-31 to 2024-03-31
┌────────────┬────────────┐
│            │ USA_Net    │
├────────────┼────────────┤
│ 2003-01-31 │ -0.0248446 │
│ 2003-02-28 │ -0.0159343 │
│ 2003-03-31 │ 0.00886423 │
│ 2003-04-30 │  0.0798764 │
│ 2003-05-31 │  0.0517369 │
│ 2003-06-30 │   0.011892 │
│ 2003-07-31 │  0.0179915 │
│ 2003-08-31 │  0.0181089 │
│     ⋮      │     ⋮      │
│ 2023-09-30 │ -0.0483353 │
│ 2023-10-31 │ -0.0235616 │
│ 2023-11-30 │  0.0895259 │
│ 2023-12-31 │  0.0456572 │
│ 2024-01-31 │  0.0151977 │
│ 2024-02-29 │  0.0518111 │
│ 2024-03-31 │  0.0309855 │
└────────────┴────────────┘
           240 rows omitted

In [127]:
# US PLN acwi_scenarios
mean_us_pln = mean(values(returns_pln[:USA_Net])) .* 12
std_us_pln = std(values(returns_pln[:USA_Net])) .* 12^0.5
println(mean_us_pln)
println(std_us_pln)
scenarios_us_pln = rand(Normal(mean_us_pln, std_us_pln), 10_000, 15)
print_percentiles(scenarios_us_pln, [0.02, 0.25, 0.5, 0.75, 0.98])

0.09847987951285313
0.14936298492263206


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.205,-0.2062,-0.2086,-0.2089,-0.2069,-0.21,-0.2003,-0.2033,-0.2176,-0.2108,-0.2122,-0.2043,-0.2078,-0.2056,-0.2118
0.25,-0.0037,-0.0016,-0.0029,0.0001,0.0002,-0.0013,-0.0006,0.0002,-0.0032,-0.0024,-0.0016,0.0003,-0.0021,-0.0014,-0.0099
0.5,0.0992,0.1004,0.0985,0.1005,0.0974,0.0993,0.1004,0.1017,0.0966,0.1011,0.0994,0.0998,0.0993,0.1018,0.0934
0.75,0.2014,0.2002,0.1998,0.1982,0.1994,0.1992,0.1999,0.2006,0.1965,0.2013,0.2006,0.2019,0.1993,0.2032,0.1955
0.98,0.4066,0.4037,0.4069,0.4043,0.4037,0.4029,0.4057,0.4075,0.4123,0.411,0.4041,0.4002,0.407,0.4075,0.4006


In [128]:
#US USD
mean_us_usd = mean(values(returns_usd[:USA_Net][Date(2003,01,01):Date(2024,03,31)])) .* 12
std_us_usd = std(values(returns_usd[:USA_Net][Date(2003,01,01):Date(2024,03,31)])) .* 12^0.5
println(mean_us_usd)
println(std_us_usd)
scenarios_us_usd = rand(Normal(mean_us_usd, std_us_usd), 10_000, 15)
print_percentiles(scenarios_us_usd, [0.02, 0.25, 0.5, 0.75, 0.98])

0.09847987951285309
0.14936298492263203


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.2109,-0.2115,-0.2053,-0.2099,-0.2047,-0.2105,-0.2117,-0.2104,-0.2016,-0.2157,-0.2085,-0.2114,-0.2082,-0.2091,-0.2067
0.25,-0.0015,-0.0054,-0.001,-0.0038,-0.0037,-0.0002,-0.0013,-0.0012,-0.0031,-0.0061,-0.0049,-0.0019,-0.0019,0.0003,-0.003
0.5,0.0993,0.0968,0.0954,0.0957,0.0984,0.1001,0.0972,0.0987,0.098,0.094,0.097,0.0976,0.1002,0.0999,0.0986
0.75,0.1999,0.1953,0.1956,0.1981,0.1987,0.1987,0.2018,0.1982,0.1997,0.1993,0.1991,0.1987,0.2013,0.1991,0.1975
0.98,0.4118,0.404,0.4088,0.4005,0.4044,0.4033,0.4157,0.4068,0.4066,0.4149,0.4114,0.4035,0.4049,0.398,0.4037


In [129]:
#USD_PLN
mean_usd = mean(values(usdpln[:USDPLN][Date(2003,01,01):Date(2024,03,31)])) .* 12
std_usd = std(values(usdpln[:USDPLN][Date(2003,01,01):Date(2024,03,31)])) .* 12^0.5
println(mean_usd)
println(std_usd)
scenarios_usd = rand(Normal(mean_usd, std_usd), 10_000, 15)
print_percentiles(scenarios_usd, [0.02, 0.25, 0.5, 0.75, 0.98])

0.001941683280832559
0.13377549414442894


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.276,-0.2729,-0.2705,-0.2715,-0.2701,-0.2753,-0.2691,-0.2704,-0.2726,-0.27,-0.2752,-0.2679,-0.2694,-0.2703,-0.2762
0.25,-0.0897,-0.087,-0.0884,-0.0907,-0.0875,-0.0871,-0.0868,-0.088,-0.0836,-0.0891,-0.0884,-0.0887,-0.0915,-0.0879,-0.0894
0.5,-0.0013,0.0018,-0.0016,-0.001,0.0003,0.0024,0.0027,0.0047,0.0073,-0.0001,0.0048,0.0002,-0.0014,0.0018,0.0016
0.75,0.0907,0.0909,0.0882,0.0902,0.0906,0.0935,0.0949,0.0931,0.0958,0.0888,0.0951,0.0919,0.0915,0.09,0.0925
0.98,0.2699,0.2693,0.282,0.2775,0.2723,0.2777,0.2754,0.2783,0.2789,0.2781,0.2761,0.2768,0.2806,0.2736,0.271


In [134]:
[values(usdpln[:USDPLN][Date(2003,01,01):Date(2024,03,31)]) values(returns_usd[:USA_Net][Date(2003,01,01):Date(2024,03,31)])]

255×2 Matrix{Float64}:
 -0.00183031  -0.0248446
  0.0232042   -0.0159343
  0.0473925    0.00886423
 -0.0730861    0.0798764
 -0.0258057    0.0517369
  0.0484517    0.011892
 -0.00834497   0.0179915
  0.0264423    0.0181089
 -0.00586097  -0.0120244
  0.0196198    0.055064
  ⋮           
 -0.0151447    0.033567
  0.0293288   -0.0175258
  0.0576333   -0.0483353
 -0.0361159   -0.0235616
 -0.0525834    0.0895259
 -0.0158441    0.0456572
  0.0175065    0.0151977
 -0.00252589   0.0518111
 -0.00107732   0.0309855

In [140]:
cov_us_usdpln = cov([values(usdpln[:USDPLN][Date(2003,01,01):Date(2024,03,31)]) values(returns_usd[:USA_Net][Date(2003,01,01):Date(2024,03,31)])])
MvNormal([mean_usd, mean_us_usd], cov_us_usdpln)
scenarios_merge = rand(MvNormal([mean_usd, mean_us_usd], cov_us_usdpln), 10_000, 15)


10000×1 Matrix{Vector{Float64}}:
 [0.3575832583622808, 1.6732334806721625]
 [0.1549972145628999, 1.1874016815996702]
 [0.05142250974786279, 1.461658805221778]
 [-0.08970842894367514, 1.650065371231431]
 [-0.12903938479427213, 1.5506294461510766]
 [0.10403576921932381, 1.6644681454997674]
 [0.12784266311465856, 1.4967154026977514]
 [-0.06060034317685287, 1.2149700254838955]
 [-0.21166566514617868, 1.3782318725839808]
 [0.07876734984065628, 1.5020811021398848]
 ⋮
 [0.0257741573486331, 1.3957332134724634]
 [0.10154216257266722, 1.3838258443297897]
 [0.1462968075231839, 1.6988051914210798]
 [0.18067599418551245, 1.5419473489568443]
 [-0.1374445342081316, 1.590133719077167]
 [0.23002793303644967, 1.5513479636382548]
 [0.014098641619554417, 1.3267242947802018]
 [-0.14558933868139035, 1.3428040592754105]
 [0.13067468849430183, 1.4739669359709842]

In [143]:
cumsum(usdpln[Date(2023,01,01):Date(2024, 03, 31)])

15×1 TimeArray{Float64, 1, Date, Vector{Float64}} 2023-01-31 to 2024-03-31
┌────────────┬─────────────┐
│            │ USDPLN      │
├────────────┼─────────────┤
│ 2023-01-31 │  -0.0103425 │
│ 2023-02-28 │    0.014964 │
│ 2023-03-31 │  -0.0151937 │
│ 2023-04-30 │  -0.0511262 │
│ 2023-05-31 │  -0.0324525 │
│ 2023-06-30 │  -0.0746935 │
│ 2023-07-31 │  -0.0898381 │
│ 2023-08-31 │  -0.0605093 │
│ 2023-09-30 │ -0.00287607 │
│ 2023-10-31 │   -0.038992 │
│ 2023-11-30 │  -0.0915754 │
│ 2023-12-31 │    -0.10742 │
│ 2024-01-31 │  -0.0899131 │
│ 2024-02-29 │  -0.0924389 │
│ 2024-03-31 │  -0.0935163 │
└────────────┴─────────────┘